In [1]:
from io import BytesIO
import os
import requests
from PIL import Image
import base64



# Load the model
load_data = {"num_gpus": 1, "models_per_gpu": 1}
response = requests.post("http://127.0.0.1:8000/load", json=load_data)

In [4]:
from io import BytesIO
import os
import requests
from PIL import Image
import base64



# Load the model
load_data = {"num_gpus": 1, "models_per_gpu": 1}
response = requests.post("http://127.0.0.1:8000/load", json=load_data)

from PIL import Image
import requests
from io import BytesIO
import os
import base64

def load_image(image_source):
    """
    Load an image from a local file path or a URL, convert it to RGB if necessary,
    and return the image as a base64-encoded JPEG.

    Args:
        image_source (str): Path to the image or URL of the image.

    Returns:
        str: Base64-encoded JPEG image.
    """
    if os.path.isfile(image_source):  # Check if the source is a local file
        image = Image.open(image_source)
    else:  # Assume the source is a URL
        response = requests.get(image_source)
        image = Image.open(BytesIO(response.content))
    
    # Convert the image to RGB mode if it is RGBA
    if image.mode == 'RGBA':
        image = image.convert('RGB')
    
    # Save the image to a buffer as a JPEG
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    
    # Encode the buffer content to base64 and return
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

# change to your image
uk = "C:/Users/Mariyan/Downloads/uk_passport.png"
# webp doesn't work
togo = "C:/Users/Mariyan/Downloads/togo_anmeldung_1.webp"
bike="https://preview.redd.it/picked-up-a-new-to-me-bike-after-a-4-year-riding-hiatus-v0-xvkmmnl8smjd1.jpeg?auto=webp&s=a0b9ac8e7c96c392d80fc0439dee06af01c63cdc"
prompt = "whats in the picture"


# Send a prompt
prompt_data = {
    "model": 0, #TODO model_path
    "image": load_image(togo), # optional, can be None
    "text": prompt
}
response = requests.post("http://127.0.0.1:8000/prompt", json=prompt_data)
print(response.json()['response'])

The image shows a document with German text, which appears to be a request for a change of residence for a person named Berlin. It includes sections for the applicant's name, address, and the reason for the change. There is also a section for the family name and surname, and a space for the date of birth. The document is stamped with the 'Bezirksamt Mitte von Berlin' and has a date of 11.04.2024. At the bottom, there is a seal with the text 'Berlin, 11.04.2024' and a logo that includes a shield with a cross and a banner with the text 'BEZIRKSAMT MITTE'.


In [2]:
import time
import requests
from PIL import ImageGrab
import base64
from io import BytesIO

# Server URL
SERVER_URL = "http://192.168.2.111:8000/prompt"

import csv
from datetime import datetime

def append_to_daily_observations(model_response: str, file_path: str = "daily_observations.txt"):
    """
    Appends the model's response to a local .txt file in CSV format, recording the current datetime and response text.

    Parameters:
    model_response (str): The text generated by the model.
    file_path (str): The path to the file where the observations will be recorded. Defaults to 'daily_observations.txt'.

    Returns:
    None
    """
    # Get the current date and time
    current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Prepare the row to be appended
    row = [current_datetime, model_response]

    # Append the row to the CSV file
    with open(file_path, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(row)


# Example usage
# model_response = "This is a sample observation from the model."
# append_to_daily_observations(" The image displays an e-meeting interface with various software applications like Explorer, Visual Studio Code, and Docker running. There's an overlay instruction for the user to analyze the current screenshot, listing elements such as applications and tools currently open, visual content, conversations and messages, task lists and reminders, and unusual elements. The user is prompted to note down insights from the analysis. The user's task analysis process includes evaluating the open applications, visual content, ongoing conversations, task lists, emails, and any out-of-the-ordinary elements to understand the current work engagement. The provided insights may indicate the user's mental and task load, potential distractions, or the alignment of their work tasks with priorities. Additionally, the user may offer suggestions or point out issues like an unaligned mood from a recent meeting, to improve the productivity or work-life balance. The key focus areas will ultimately help tailor personal productivity enhancing strategies.")

def take_screenshot():
    """Take a screenshot of the current screen."""
    screenshot = ImageGrab.grab()
    return screenshot

def encode_image_to_base64(image):
    """Convert the screenshot to base64."""
    # Convert RGBA to RGB if necessary
    if image.mode == 'RGBA':
        image = image.convert('RGB')

    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

prompt = '''
You are Mariyan, as you analyze this current screenshot, focus on the information and 
tasks directly in front of you. What does this snapshot reveal about your current 
state of mind, priorities, or focus? Consider the applications, documents, or 
conversations that are open on your screen right now. What do they suggest about 
your immediate goals or concerns? Provide insights on how this moment fits into 
the larger picture of your day, but emphasize the unique aspects of what’s 
happening right now. What can you learn or infer from this specific moment?
'''

structured = '''

image_context : 
main_activity:

'''

focused = '''
Mariyan, as you analyze this current image of your daily life, focus on extracting the most curious and insightful information from what’s in front of you. Pay close attention to:

	1.	Applications and Tools: What applications or tools are currently open? What does their presence suggest about your current tasks, priorities, or state of mind? Are these tools aligned with your immediate goals or do they hint at a distraction?
	2.	Visual Content: If there are photos or non-text visuals, what do they depict? How do these images relate to your day-to-day life, and what might they reveal about your personal interests or current focus areas?
	3.	Conversations and Messages: Are there any open chat windows, emails, or other forms of communication? What do these interactions suggest about your current social or professional engagements? What tone or themes are evident in these conversations?
	4.	Task Lists and Reminders: If there are any visible to-do lists, reminders, or schedules, what do they indicate about your immediate priorities or concerns? Are there any overdue tasks or upcoming deadlines that stand out?
	5.	Unusual Elements: Identify anything that seems out of place or particularly interesting. What could these anomalies reveal about your current environment, mindset, or recent activities?

Based on the analysis, provide a summary of the insights that can be drawn from this moment. How do these elements fit together, and what can you infer about your current state of mind, focus, or goals?

'''
def send_screenshot(image_base64):
    """Send the base64-encoded screenshot to the server."""
    payload = {
        "model": 0,
        "image": image_base64,
        "text": "what is this",
        "audio": None,
        "model_name": "phi"
    }
    try:
        response = requests.post(SERVER_URL, json=payload)
    except Exception as e:
        print("Error sending screenshot:", e.content)

    print(response.json()['response'])

    return f"{response.json()}"

def send_audio(audio: str):
    """Send the base64-encoded audio to the server."""
    payload = {
        "model": 1,
        "image": None,
        "text": "Summarise this",
        "audio": audio,
        "model_name": "whisper"
    }
    try:
        response = requests.post(SERVER_URL, json=payload)
    except Exception as e:
        print("Error sending screenshot:", e)

    print(response.json()['response'])

    return f"{response.json()}"


while True:
        # IDEA: add a keyboard shortcut for highlight moment    
        # Take a screenshot
        screenshot = take_screenshot()
        
        # Encode the image to base64
        image_base64 = encode_image_to_base64(screenshot)
        
        # Send the image to the server
        response = send_screenshot(image_base64)
        
        append_to_daily_observations(response)
        
        # Wait for n minutes between screenshots
        n = 1
        time.sleep(60*n)

OSError: screen grab failed

In [6]:
import pyaudio
import wave
import numpy as np
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import datetime

def is_silent(audio_segment, silence_thresh=-50.0, min_silence_len=3000):
    """Return True if the given audio segment is silent."""
    non_silent_ranges = detect_nonsilent(audio_segment, min_silence_len=min_silence_len, silence_thresh=silence_thresh)
    return len(non_silent_ranges) == 0

def record_audio_on_sound(threshold=500, chunk_size=1024, sample_format=pyaudio.paInt16, channels=1, rate=44100, silence_duration=2):
    p = pyaudio.PyAudio()

    # List all audio devices and find BlackHole
    for i in range(p.get_device_count()):
        dev_info = p.get_device_info_by_index(i)
        if "BlackHole" in dev_info['name']:
            device_index = i
            break
    else:
        raise ValueError("BlackHole device not found.")

    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=rate,
                    input=True,
                    input_device_index=device_index,
                    frames_per_buffer=chunk_size)

    print("Listening for sound...")
    frames = []
    recording = False
    silence_frames = 0

    sample_width = p.get_sample_size(sample_format)

    try:
        while True:
            data = stream.read(chunk_size)
            audio_segment = AudioSegment(data, sample_width=sample_width, frame_rate=rate, channels=channels)

            # Convert data to numpy array for processing
            audio_data = np.frombuffer(data, dtype=np.int16)
            if np.abs(audio_data).mean() > threshold:
                if not recording:
                    print("Sound detected, starting recording...")
                    recording = True
                frames.append(data)
                silence_frames = 0
            elif recording:
                silence_frames += 1
                if silence_frames >= silence_duration * (rate // chunk_size):
                    print("Silence detected, stopping recording...")
                    break
                frames.append(data)
    
    except KeyboardInterrupt:
        pass

    stream.stop_stream()
    stream.close()
    p.terminate()

    if recording:
        filename = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + "_recording.wav"
        wf = wave.open(filename, 'wb')
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        wf.writeframes(b''.join(frames))
        wf.close()
        print(f"Recording saved to {filename}")
    else:
        print("No sound detected, no recording made")

if __name__ == "__main__":
    record_audio_on_sound()

Listening for sound...
Sound detected, starting recording...
Silence detected, stopping recording...
Recording saved to 2024-08-21_14-43-39_recording.wav


In [3]:
import base64

def audio_file_to_str_bytes(file_path: str) -> str:
    """
    Convert an audio file to a base64 encoded string.
    
    Parameters:
    file_path (str): Path to the audio file.
    
    Returns:
    str: The base64 encoded string representation of the audio file.
    """
    with open(file_path, "rb") as audio_file:
        audio_data = audio_file.read()
        encoded_audio = base64.b64encode(audio_data).decode('utf-8')
    return encoded_audio

# Example usage
audio_str = audio_file_to_str_bytes("C:\\Users\\Mariyan\\dev\\model_os\\decode_test.wav")
print(audio_str)

send_audio(audio_str)

...


NameError: name 'send_audio' is not defined

In [4]:
import time
import requests
from PIL import ImageGrab
import base64
from io import BytesIO
import numpy as np

def send_audio(audio: np.ndarray):
    """Send the base64-encoded audio to the server."""

      # Convert the NumPy array (audio data) to bytes
    audio_bytes = audio.tobytes()
    
    # Encode the bytes to base64 to make it serialisable
    audio_base64 = base64.b64encode(audio_bytes).decode('utf-8')

    payload = {
        "model": 0,
        "image": None,
        "text": "Summarise this",
        "audio": audio_base64,
        "model_name": "phi"
    }
    try:
        response = requests.post('http://127.0.0.1:8000/prompt', json=payload)
        print(response.json())  # Handle the response from the server
    except TypeError as e:
        print("Error sending audio:", e)
    except requests.exceptions.RequestException as e:
        print("Request failed:", e)

    # print(response.json()['response'])

    return f"{response.json()}"


# import base64

# def audio_file_to_str_bytes(file_path: str) -> str:
#     """
#     Convert an audio file to a base64 encoded string.
    
#     Parameters:
#     file_path (str): Path to the audio file.
    
#     Returns:
#     str: The base64 encoded string representation of the audio file.
#     """
#     with open(file_path, "rb") as audio_file:
#         audio_data = audio_file.read()
#         encoded_audio = base64.b64encode(audio_data).decode('utf-8')
#     return encoded_audio

# # Example usage
# audio_str = audio_file_to_str_bytes("C:\\Users\\Mariyan\\dev\\2024-08-21_14-43-39_recording.wav")
# print(audio_str)

# # send_audio(audio_str)

# import wave

import wave
import os

import numpy as np

def get_wav_sampling_rate(file_path):
    # Check if the file exists
    if not os.path.isfile(file_path):
        print(f"File {file_path} does not exist.")
        return None
    
    try:
        with wave.open(file_path, 'rb') as wav_file:
            sampling_rate = wav_file.getframerate()
            return sampling_rate
    except wave.Error as e:
        print(f"Failed to read {file_path}: {e}")
        return f"Failed to read {file_path}: {e}"

# wav_file_path="C:\\Users\\Mariyan\\dev\\2024-08-21_14-43-39_recording.wav"
# print(get_wav_sampling_rate("C:\\Users\\Mariyan\\dev\\2024-08-21_14-43-39_recording.wav"))

# Function to load the WAV file and extract audio data
def load_wav(file_path):
    with wave.open(file_path, 'rb') as wav_file:
        # Read the WAV file properties
        sample_rate = wav_file.getframerate()
        n_frames = wav_file.getnframes()
        # Extract audio data
        audio_data = wav_file.readframes(n_frames)
        audio_array = np.frombuffer(audio_data, dtype=np.int16).astype(np.float32) / 32768.0  # Convert to float32 array
    return audio_array, sample_rate
    
# Load the WAV file
audio_array, sampling_rate = load_wav("/Users/m/Downloads/harvard.wav")

send_audio(audio_array)



{'response': "I'm sorry, but I'm not sure what you're asking me to summarize. Could you please provide more context or clarify your request?"}


'{\'response\': "I\'m sorry, but I\'m not sure what you\'re asking me to summarize. Could you please provide more context or clarify your request?"}'